In [1]:
import csv
import pandas as pd

from datetime import date
from datetime import datetime

today = date.today()
now = datetime.now()
iday = today.strftime("%y%m%d")
itime = now.strftime("%H%M")

In [2]:
def write_transaction(count, patient_last, patient_first, patient_id, jail_name, jail_city, jail_state, jail_zip, 
                      patient_birthday, consult_id, responded_date, icd10_code, pcp_last,pcp_first, pcp_npi, sr_last, sr_first, sr_npi):
    transaction = 'ST*837*{}*005010X222A1~\n'.format(count) +\
    'BHT*0019*00*0123*20{}*{}*CH~\n'.format(iday,itime)+\
    'NM1*41*2*HUBMD PC*****46*1982269031~\n'+\
    'PER*IC*Waheed Baqai*TE*9094559473~\n'+\
    'NM1*40*2*CCIH*****46*CCIH~\n'+\
    'HL*1**20*1~\n'+\
    'PRV*BI*PXC*193200000X~\n'+\
    'NM1*85*2*HUBMD PC*****XX*1982269031~\n'+\
    'N3*104 E Olive Ave Ste 100~\n'+\
    'N4*REDLANDS*CA*92373-5255~\n'+\
    'REF*EI*832774928~\n'+\
    'PER*IC*Waheed Baqai*TE*9094559473~\n'+\
    'HL*2*1*22*0~\n'+\
    'SBR*P*18*******MC~\n'+\
    'NM1*IL*1*{}*{}****MI*{}~\n'.format(patient_last, patient_first, patient_id)+\
    'N3*{}~\n'.format(jail_name)+\
    'N4*{}*{}*{}~\n'.format(jail_city, jail_state, jail_zip)+\
    'DMG*D8*{}*U~\n'.format(patient_birthday)+\
    'NM1*PR*2*CCIH*****PI*CCIH~\n'+\
    'CLM*{}*95***02:B:1*Y*A*Y*Y~\n'.format(consult_id)+\
    'DTP*304*D8*{}~\n'.format(responded_date)+\
    'HI*ABK:{}~\n'.format(icd10_code)+\
    'NM1*DN*1*{}*{}****XX*{}~\n'.format(pcp_last,pcp_first, pcp_npi) +\
    'NM1*82*1*{}*{}****XX*{}~\n'.format(sr_last, sr_first, sr_npi)+\
    'LX*1~\n'+\
    'SV1*HC:99451:GQ*95*UN*1*02**1~\n'+\
    'DTP*472*D8*{}~\n'.format(responded_date)+\
    'SE*28*{}~\n'.format(count)

    return transaction

In [3]:
import pandas as pd
import os 

os.chdir('C:/Users/Melton/Box/Afya Dashboard')
df_afya_dashboard = pd.read_excel('Source for Afya Earnings Dashboard.xlsx', engine = 'openpyxl') 

In [4]:
df_afya_dashboard.columns

Index(['Provider NPI', 'Response Time (Hours)', 'Response Time (Days)',
       'C_RESPONDED', 'Network', 'SR_NAME', 'Consult ID', 'First Name',
       'Last Name', 'CDC#', 'DOB', 'Diagnosis Code', 'Created', 'Created By',
       'PCPFirst Name', 'PCPLast Name', 'PCP NPI', 'Referring Site Address',
       'Referring Site Tax ID', 'SRFirst Name', 'SRLast Name', 'SR NPI',
       'SR Org', 'Specialty', 'Closed Date', 'Close Code', '$ Amount'],
      dtype='object')

In [5]:
responded_arr = df_afya_dashboard['C_RESPONDED'].to_numpy()
print(len(responded_arr))

108780


In [6]:
import numpy as np

this_month_indices = []
responded_arr = df_afya_dashboard['C_RESPONDED'].to_numpy()
for date in responded_arr:
    try:
        month = date[:date.find('/')]
        nomonth = date[date.find('/')+1:]
        day = nomonth[:nomonth.find('/')]
        noday = nomonth[nomonth.find('/')+1:]
        year = noday[:noday.find(' ')]
        int_day = int(day)
        if( year == '2021' and (month == '3' or month == '03') and int_day < 15):
            index = np.where(responded_arr==date)[0][0]
            if( df_afya_dashboard['Network'][index]=='CDCR' ):
                this_month_indices.append(index)
    except:
        pass

In [7]:
print(len(this_month_indices))

113


In [8]:
import numpy as np

transactions = []
counter_int = 1
for ind in this_month_indices:
    data = df_afya_dashboard.iloc[ind]
    pt_last = data['Last Name']
    pt_first = data['First Name']
    pt_id = data['CDC#']

    #Jail Stuff
    full_address = data['Referring Site Address']

    if(full_address.find('(') == -1):
        jail_street = full_address[:full_address.find(',')]
        temp_address = full_address[(full_address.find(',')+2):]
        jail_city = temp_address[:temp_address.index(',')]
        temp_address = temp_address[(temp_address.index(',')+2):]
        jail_zip = temp_address[(temp_address.index(',')+2):]

    jail_state = 'CA'

    _dob = data['DOB']
    
    month = _dob[:_dob.find('/')]
    nomonth = _dob[_dob.find('/')+1:]
    day = nomonth[:nomonth.find('/')]
    year = nomonth[nomonth.find('/')+1:]
    dob = year+month.zfill(2)+day.zfill(2)
    
    eConsultID = data['Consult ID'].astype(np.int64)
    icd = data['Diagnosis Code'].replace('>', '').replace('<','')

    day = data['C_RESPONDED']

    month = day[:day.find('/')]
    nomonth = day[day.find('/')+1:]
    day = nomonth[:nomonth.find('/')]
    noday = nomonth[nomonth.find('/')+1:]
    year = noday[:noday.find(' ')]
    responded_day = year+month.zfill(2)+day.zfill(2)

    sr_last = data["SRLast Name"]
    sr_first = data["SRFirst Name"]
    sr_npi = data["SR NPI"].astype(np.int64)
    
    pcp_last = data['PCPFirst Name']
    pcp_first = data['PCPLast Name']
    pcp_npi = data['PCP NPI']
    
    counter_str = str(counter_int)
    transactions.append(write_transaction(counter_str.zfill(4), pt_last, pt_first, pt_id, jail_street, jail_city, jail_state, jail_zip, dob, eConsultID, responded_day, icd, pcp_last, pcp_first, pcp_npi, sr_last, sr_first, sr_npi))
    counter_int += 1

In [9]:
print(transactions[100])

ST*837*0101*005010X222A1~
BHT*0019*00*0123*20210407*1258*CH~
NM1*41*2*HUBMD PC*****46*1982269031~
PER*IC*Waheed Baqai*TE*9094559473~
NM1*40*2*CCIH*****46*CCIH~
HL*1**20*1~
PRV*BI*PXC*193200000X~
NM1*85*2*HUBMD PC*****XX*1982269031~
N3*104 E Olive Ave Ste 100~
N4*REDLANDS*CA*92373-5255~
REF*EI*832774928~
PER*IC*Waheed Baqai*TE*9094559473~
HL*2*1*22*0~
SBR*P*18*******MC~
NM1*IL*1*HINSON*WILLIAM****MI*BJ0577~
N3*SQ~
N4*San Quentin*CA*94964~
DMG*D8*19770516*U~
NM1*PR*2*CCIH*****PI*CCIH~
CLM*8883849*95***02:B:1*Y*A*Y*Y~
DTP*304*D8*20210312~
HI*ABK:D22.9~
NM1*DN*1*John*Grant****XX*1326112632~
NM1*82*1*Herriford*Carla****XX*1114910163~
LX*1~
SV1*HC:99451:GQ*95*UN*1*02**1~
DTP*472*D8*20210312~
SE*28*0101~



In [10]:
from datetime import date
from datetime import datetime
from random import uniform
today = date.today()
now = datetime.now()
iday = today.strftime("%y%m%d")
itime = now.strftime("%H%M")
num_of_transactions = len(transactions)
interchange_control_number = str(int(uniform(1,999999999))).zfill(9)
group_control_number = str(int(uniform(1,999999999))).zfill(9)

In [11]:
header_lines = 'ISA*00*          *00*          *ZZ*AV09311993     *01*030240928      *{}*{}*^*00501*{}*1*P*:~\n'.format(iday,itime,interchange_control_number)+\
'GS*HC*AV01101957*030240928*20{}*{}*{}*X*005010X222A1~\n'.format(iday, itime, group_control_number)


In [12]:
for trans_ in transactions:
    header_lines += trans_

In [13]:
trailer_lines = 'GE*{}*{}~\n'.format(num_of_transactions, group_control_number)+\
'IEA*1*{}~\n'.format(interchange_control_number) 

In [14]:
header_lines+=trailer_lines

In [15]:
print(header_lines)

ISA*00*          *00*          *ZZ*AV09311993     *01*030240928      *210407*1300*^*00501*363596845*1*P*:~
GS*HC*AV01101957*030240928*20210407*1300*340075674*X*005010X222A1~
ST*837*0001*005010X222A1~
BHT*0019*00*0123*20210407*1258*CH~
NM1*41*2*HUBMD PC*****46*1982269031~
PER*IC*Waheed Baqai*TE*9094559473~
NM1*40*2*CCIH*****46*CCIH~
HL*1**20*1~
PRV*BI*PXC*193200000X~
NM1*85*2*HUBMD PC*****XX*1982269031~
N3*104 E Olive Ave Ste 100~
N4*REDLANDS*CA*92373-5255~
REF*EI*832774928~
PER*IC*Waheed Baqai*TE*9094559473~
HL*2*1*22*0~
SBR*P*18*******MC~
NM1*IL*1*CANALES*OSMAN****MI*AN2588~
N3*SQ~
N4*San Quentin*CA*94964~
DMG*D8*19810722*U~
NM1*PR*2*CCIH*****PI*CCIH~
CLM*8882790*95***02:B:1*Y*A*Y*Y~
DTP*304*D8*20210310~
HI*ABK:M25.541~
NM1*DN*1*John*Grant****XX*1326112632~
NM1*82*1*Frencher*James****XX*1821007394~
LX*1~
SV1*HC:99451:GQ*95*UN*1*02**1~
DTP*472*D8*20210310~
SE*28*0001~
ST*837*0002*005010X222A1~
BHT*0019*00*0123*20210407*1258*CH~
NM1*41*2*HUBMD PC*****46*1982269031~
PER*IC*Waheed Baqai*T

In [16]:
os.chdir('C:/Users/Melton/OneDrive/Documents/Afya/Claims')
f = open("march_claims_P.edi", "w")
f.write(header_lines)
f.close()